<a href="https://colab.research.google.com/github/ClementineDomine/NeuralPlayground/blob/comparison_board/colab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeuralPlayground Introduction